In [2]:
import cantera as ct
import optunity
import pandas as pd
import re
import os

def shift_nasa(phase, index, change_in_enthalpy):        
    
    if phase.molecular_weights[index]>min(phase.molecular_weights):        
        my_species=phase.species(index)
        
        new_coeffs=my_species.thermo.coeffs
        T_low=my_species.thermo.min_temp
        T_high=my_species.thermo.max_temp
        
        change_in_enthalpy/=ct.gas_constant
        new_coeffs[6]=my_species.thermo.coeffs[6] - change_in_enthalpy
        new_coeffs[13]=my_species.thermo.coeffs[13] - change_in_enthalpy
        
        my_species.thermo=ct.NasaPoly2(T_low, T_high,ct.one_atm, new_coeffs)
        phase.modify_species(index,my_species)
             
    return   

def shift_Arrhenius(phase, index, change_in_activation_energy):
    """Use this subroutine to update the Arrhenius coefficients for a reaction, given by index. 
    It takes as input:
    (i) an index for the reaction of interest, 
    (ii) the change in the activation energy, and
    #(iii) the change in the prefactor.
    NOTE: the change in the activation energy must be in J/kmol!
    NOTE: the change in prefactor is only used for sticking coefficients"""
    # create a copy of the reaction
    my_rxn = phase.reaction(index)
    newA = my_rxn.rate.pre_exponential_factor
    newb = my_rxn.rate.temperature_exponent
    newEa = my_rxn.rate.activation_energy
    
    #for sticking coefficients, we don't need/want to change the activation energy
    if my_rxn.reaction_type == 'sticking-Arrhenius':
        pass
    else:
        newEa += change_in_activation_energy
        
    new_rate = ct.InterfaceArrheniusRate(A=newA, b=newb, Ea = newEa)
    my_rxn.rate = new_rate
    phase.modify_reaction(index, my_rxn)
    
    return


def create_new_mechanism(phase, sample):
    #cycle through the species for each phase
    #cycle through the reactions for each phase 
    
    if phase == surf111:
        facet_dir = '../DFT/ensembles/Ni111/'
    elif phase == surf211:
        facet_dir = '../DFT/ensembles/Ni211/'
    elif phase == surf100:
        facet_dir = '../DFT/ensembles/Ni100/'
    elif phase == surf110:
        facet_dir = '../DFT/ensembles/Ni110/'
        
    for i in range(phase.n_species):        
        if phase.molecular_weights[i]>min(phase.molecular_weights):        
            my_species=phase.species(i)
            
            if my_species.name.startswith('CO2('):
                filename='CO2X_19_bee.txt'
            elif my_species.name.startswith('OC('):
                filename='XCO_19_bee.txt'            
            elif my_species.name.startswith('O('):
                filename='XO_19_bee.txt'
            elif my_species.name.startswith('C('):
                filename='XC_19_bee.txt'
        
            file_path = os.path.join(facet_dir, filename)
        
            bee=pd.read_csv(file_path, sep="\t", header=0)
    
            Delta_dH = bee['delta_dH'][sample]*1e6       

            shift_nasa(phase, i, Delta_dH)
        
    for i in range(phase.n_reactions):
        my_rxn = phase.reaction(i)
        
        if my_rxn.reaction_type == 'sticking-Arrhenius':
            pass
        else:
            #Use a regular expression to remove all parts enclosed by parentheses
            cleaned_reaction = re.sub(r'\([^)]*\)', '', str(my_rxn))
            if cleaned_reaction == 'OC + site <=> C + O':
                ea_filename='C-O_bee.txt'
            elif cleaned_reaction == 'O + OC <=> CO2 + site':
                ea_filename='OC-O-rev_bee.txt'
        
            ea_file_path = os.path.join(facet_dir, ea_filename)
        
            ea_bee=pd.read_csv(ea_file_path, sep="\t", header=0)
            Delta_Ea = ea_bee['delta_Ea'][sample]*1e6
            
            shift_Arrhenius(phase, i, Delta_Ea)
            
    return

num_pts = 10 # the number of points to generate 
start=0

directory='mechanisms/File_'

for k in range(num_pts):
    
    #Create a new folder for the new input file and write the input file
    mkdir_writer="".join((directory,str(k+start)))
    os.mkdir(mkdir_writer)    
    
    #Some parameters
    T=298 #Initial temperature
    p=1 #pressure 
    # input file containing the surface reaction mechanism
    cti_file = 'multifacet_original.yaml'
    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, CO2:0.0, CO:0'
    
    # import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0, 'CO2(111)':0}    

    # import the (211) surface
    surf211 = ct.Interface(cti_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1 , 'O(211)':0.0, 'OC(211)':0.0, 'CO2(211)':0}

    # import the (100) surface
    surf100 = ct.Interface(cti_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1 , 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}
    
    # import the (110) surface
    surf110 = ct.Interface(cti_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1 , 'O(110)':0.0, 'OC(110)':0.0, 'CO2(110)':0}
    
    create_new_mechanism(surf111,k)
    create_new_mechanism(surf211,k)    
    create_new_mechanism(surf100,k)
    create_new_mechanism(surf110,k)
        
    surf111.write_yaml(mkdir_writer + '/test111.yaml')
    surf211.write_yaml(mkdir_writer + '/test211.yaml')
    surf100.write_yaml(mkdir_writer + '/test100.yaml')
    surf110.write_yaml(mkdir_writer + '/test110.yaml')


In [88]:
T=25 #Initial temperature
p=1 #pressure 
# input file containing the surface reaction mechanism
cti_file = 'test.yaml'
    
# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')
gas.TPX = T, p*ct.one_atm, 'Ar:1, CO2:0.0, CO:0'
    
# import the (111) surface
surf111 = ct.Interface(cti_file,'surface1', [gas])
surf111.TP = T, p*ct.one_atm
surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0, 'CO2(111)':0}

# import the (211) surface
surf211 = ct.Interface(cti_file,'surface2', [gas])
surf211.TP = T, p*ct.one_atm
surf211.coverages = {'site(211)':1 , 'O(211)':0.0, 'OC(211)':0.0, 'CO2(211)':0}

# import the (100) surface
surf100 = ct.Interface(cti_file,'surface3', [gas])
surf100.TP = T, p*ct.one_atm
surf100.coverages = {'site(100)':1 , 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

# import the (110) surface
surf110 = ct.Interface(cti_file,'surface4', [gas])
surf110.TP = T, p*ct.one_atm
surf110.coverages = {'site(110)':1 , 'O(110)':0.0, 'OC(110)':0.0, 'CO2(110)':0}

print(surf111.standard_enthalpies_RT*ct.gas_constant*surf111.T/1e6)


[   0.         -234.09020744 -238.2002348   157.50280679 -421.27971986]
['O2 + 2 site(111) <=> 2 O(111)', 'CO2 + site(111) <=> CO2(111)', 'CO + site(111) <=> OC(111)', 'O(111) + OC(111) <=> CO2(111) + site(111)', 'OC(111) + site(111) <=> C(111) + O(111)']
['O2 + 2 site(211) <=> 2 O(211)', 'CO2 + site(211) <=> CO2(211)', 'CO + site(211) <=> OC(211)', 'O(211) + OC(211) <=> CO2(211) + site(211)', 'OC(211) + site(211) <=> C(211) + O(211)']
